In [1]:
import bs_functions as bs_model
import numpy as np
from scipy.stats import norm
import datetime as dt
from datetime import datetime
from yahoo_fin import options
from yahoo_fin import stock_info
import yfinance as yf
import datetime as dt
from datetime import datetime

### Example: Valuation of a Microsoft call option

Strike Price: 240$

Expiration Date: 07/07/2023

#### 1. Initializing parameters

In [2]:
# Estimating Volatility by using 60-Day Historical Volatility

end = dt.datetime(2023,6,2) #dt.datetime.now()
start = end-dt.timedelta(days=60)

df = stock_info.get_data("msft", start, end)

simple_returns = df.close.pct_change().dropna()
log_returns = np.log(simple_returns + 1)

anual_volatility = log_returns.std()* np.sqrt(252)

In [3]:
K = 240

S = yf.download(tickers='msft', period='1m').Close[0]

r = 0.0361

expiration_date = dt.datetime(2023,7,7)
t_mat = expiration_date - datetime.now()
T = t_mat.days/365

sigma = anual_volatility

[*********************100%***********************]  1 of 1 completed


#### 2. Option evaluation and risk assessment

In [6]:
print("option value: \t" + str(bs_model.option_price(r,S,K,T,sigma,"CALL")))
print("Delta: \t\t" + str(bs_model.delta(r,S,K,T,sigma,"CALL")))
print("Gamma: \t\t" + str(bs_model.gamma(r,S,K,T,sigma,"CALL")))
print("Theta: \t\t"+ str(bs_model.theta(r,S,K,T,sigma,"CALL")))
print("Vega: \t\t" + str(bs_model.vega(r,S,K,T,sigma,"CALL")))
print("Rho: \t\t" + str(bs_model.rho(r,S,K,T,sigma,"CALL")))

option value: 	96.1584561739947
Delta: 		0.9999867522861836
Gamma: 		2.1493455101292865e-06
Theta: 		-0.023686436959219053
Vega: 		5.8127188678003106e-05
Rho: 		0.20974211019340844


#### 3. Comparison with actual price

In [5]:
options.get_options_chain("msft", expiration_date)["calls"].head()

,Contract Name,Last Trade Date,Strike,Last Price,Bid,Ask,Change,% Change,Volume,Open Interest,Implied Volatility
0,MSFT230707C00240000,2023-05-26 3:48PM EDT,240.0,94.20,95.80,97.35,0.00,-,1,1,66.33%
1,MSFT230707C00260000,2023-06-01 3:55PM EDT,260.0,74.08,75.85,77.70,0.00,-,3,10,54.81%
2,MSFT230707C00280000,2023-06-02 12:19PM EDT,280.0,55.45,55.95,58.85,3.06,+5.84%,1,2,53.80%
3,MSFT230707C00285000,2023-06-01 10:07AM EDT,285.0,45.10,51.35,53.70,0.00,-,6,18,49.29%
4,MSFT230707C00290000,2023-06-01 2:13PM EDT,290.0,43.85,46.65,48.30,0.00,-,3,4,43.67%
